# Initialise Firebase Application

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate(
    "ai-formative-assessments-firebase-adminsdk-qlzlv-0941d85561.json"
)
app = firebase_admin.initialize_app(cred)
db = firestore.client()

# Read Data From Excel File

In [2]:
import pandas as pd
import json

In [3]:
xl = pd.ExcelFile("question_list.xlsx")
df = xl.parse("Bio")

questions = df["Question"]
marking_schemes = df["Marking Scheme"]
data = []

for question, marking_scheme in zip(questions, marking_schemes):
    data_dict = {}
    data_dict["question"] = question
    data_dict["marking_scheme"] = marking_scheme.split("\n")
    data.append(data_dict)

data

[{'question': 'The rate of photosynthesis is affected by the colour of light. The colour of light is a limiting factor. \n\nUsing the colour of light, explain the term limiting factor.',
  'marking_scheme': ['The rate of photosynthesis will increase or decrease ',
   'when there is a change in the colour of light used.']},
 {'question': 'Describe how urea in the blood enters the nephrons and is removed through the renal pelvis.',
  'marking_scheme': ['The diameter of the lumen of the afferent arteriole is larger than the efferent arteriole. This generates a high hydrostatic pressure at the glomerulus',
   'Urea is forced through the glomerulus wall and basement membrane by ultra-filtration',
   'are collected in the bowman’s capsule and collecting duct.']},
 {'question': 'Adam’s heart rate increased to 160 bpm during his sprint to the car park after hearing the fire alarm. \n\nSuggest two body responses not mentioned in the passage that occurs as a result of the stimulus.',
  'marking_

In [4]:
with open("question_list.json", "w") as f:
    json.dump(data, f)

# Push Data to Firebase

In [5]:
data_proc = {}

for i, data_dict in enumerate(data):
    marking_scheme_dict = {}

    for j, marking_point in enumerate(data_dict["marking_scheme"]):
        marking_scheme_dict[str(j)] = marking_point

    data_proc[str(i)] = data_dict.copy()
    data_proc[str(i)]["marking_scheme"] = marking_scheme_dict

data_proc

{'0': {'question': 'The rate of photosynthesis is affected by the colour of light. The colour of light is a limiting factor. \n\nUsing the colour of light, explain the term limiting factor.',
  'marking_scheme': {'0': 'The rate of photosynthesis will increase or decrease ',
   '1': 'when there is a change in the colour of light used.'}},
 '1': {'question': 'Describe how urea in the blood enters the nephrons and is removed through the renal pelvis.',
  'marking_scheme': {'0': 'The diameter of the lumen of the afferent arteriole is larger than the efferent arteriole. This generates a high hydrostatic pressure at the glomerulus',
   '1': 'Urea is forced through the glomerulus wall and basement membrane by ultra-filtration',
   '2': 'are collected in the bowman’s capsule and collecting duct.'}},
 '2': {'question': 'Adam’s heart rate increased to 160 bpm during his sprint to the car park after hearing the fire alarm. \n\nSuggest two body responses not mentioned in the passage that occurs as

In [6]:
doc_ref = db.collection("example").document("questions_list")
doc_ref.set(data_proc)

update_time {
  seconds: 1710789347
  nanos: 197888000
}

# Pull Data From Firebase

In [7]:
doc_ref = db.collection("example").document("questions_list")
doc = doc_ref.get()
if doc.exists:
    doc_dict = doc.to_dict()

In [8]:
doc_dict

{'0': {'question': 'The rate of photosynthesis is affected by the colour of light. The colour of light is a limiting factor. \n\nUsing the colour of light, explain the term limiting factor.',
  'marking_scheme': {'0': 'The rate of photosynthesis will increase or decrease ',
   '1': 'when there is a change in the colour of light used.'}},
 '7': {'question': 'Compare the structures of the nucleus and mitochondrion.',
  'marking_scheme': {'0': 'Both nucleus and mitochondrion are enclosed by a double membrane',
   '3': 'Nucleus has nuclear pores in its nuclear envelope, while mitochondrion does not have pores in its double membrane;',
   '1': 'Both contain DNA;',
   '2': 'Mitochondrion has a highly folded inner membrane forming cristae, while the inner nuclear membrane is not folded;'}},
 '9': {'question': 'Describes the importance of diffusion in nutrient uptake and gaseous exchange in plants and humans',
  'marking_scheme': {'0': 'For nutrient uptake in humans, diffusion allows digested 